# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-01-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-01-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-02 05:25:04,33.93911,67.709953,41501,1536,34326,5639.0,Afghanistan,106.608704,3.701116
1,NaN,NaN,NaN,Albania,2020-11-02 05:25:04,41.15330,20.168300,21202,518,11246,9438.0,Albania,736.743346,2.443166
2,NaN,NaN,NaN,Algeria,2020-11-02 05:25:04,28.03390,1.659600,58272,1973,40395,15904.0,Algeria,132.886235,3.385846
3,NaN,NaN,NaN,Andorra,2020-11-02 05:25:04,42.50630,1.521800,4825,75,3475,1275.0,Andorra,6244.742121,1.554404
4,NaN,NaN,NaN,Angola,2020-11-02 05:25:04,-11.20270,17.873900,11035,286,4920,5829.0,Angola,33.575458,2.591754


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,40687,40768,40833,40937,41032,41145,41268,41334,41425,41501
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,18556,18858,19157,19445,19729,20040,20315,20634,20875,21202
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,55630,55880,56143,56419,56706,57026,57332,57651,57942,58272


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1507,1511,1514,1518,1523,1529,1532,1533,1536,1536
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,469,473,477,480,487,493,499,502,509,518
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1897,1907,1914,1922,1931,1941,1949,1956,1964,1973


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34010,34023,34129,34150,34217,34237,34239,34258,34321,34326
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,10466,10548,10654,10705,10808,10893,11007,11097,11189,11246
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,38788,38932,39095,39273,39444,39635,39635,40014,40201,40395


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2030,2048,2059,2074,2082,2103,2126,2141,2159,2173
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6615,6637,6658,6694,6712,6743,6768,6888,6940,6966
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1012,1031,1033,1033,1042,1045,1055,1056,1060,1061


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,30,31,31,31,31,31,31,31,31,31
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,69,69,69,69,69,69,69,71,71,71
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
634,1001.0,Autauga,Alabama,US,2020-11-02 05:25:04,32.539527,-86.644082,2173,31,0,2142.0,"Autauga, Alabama, US",3889.455691,1.426599
671,1075.0,Lamar,Alabama,US,2020-11-02 05:25:04,33.779950,-88.096680,534,6,0,528.0,"Lamar, Alabama, US",3868.163709,1.123596
672,1077.0,Lauderdale,Alabama,US,2020-11-02 05:25:04,34.901719,-87.656247,2743,43,0,2700.0,"Lauderdale, Alabama, US",2958.082153,1.567627


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,9206975,230995,3630579
India,8229313,122607,7544798
Brazil,5545705,160074,5022338


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,9206975,230995,3630579,5345401,2020-11-02 05:25:04,37.936303,-91.379001
India,8229313,122607,7544798,561908,2020-11-02 05:25:04,23.088275,81.806127
Brazil,5545705,160074,5022338,363293,2020-11-02 05:25:04,-12.669522,-48.480493
Russia,1624648,28026,1218619,378003,2020-11-02 05:25:04,54.546312,62.120860
France,1458999,37057,123664,1298278,2020-11-02 05:25:04,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,936816,18502,0
California,935878,17671,0
Florida,807412,16789,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,936816,18502,0,918314,2020-11-02 05:25:04,31.660643,-98.653069
California,935878,17671,0,918207,2020-11-02 05:25:04,37.843962,-120.728594
Florida,807412,16789,0,790623,2020-11-02 05:25:04,28.940755,-82.700744
New York,509735,33535,0,476200,2020-11-02 05:25:04,42.544151,-75.474183
Illinois,423527,10075,0,413452,2020-11-02 05:25:04,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,309190,7074,0
Illinois,Cook,193102,5500,0
Florida,Miami-Dade,186809,3662,0
Texas,Harris,162306,2811,0
Arizona,Maricopa,159781,3606,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,309190,7074,0,302116,2020-11-02 05:25:04,34.308284,-118.228241,6037.0
Illinois,Cook,193102,5500,0,187602,2020-11-02 05:25:04,41.841448,-87.816588,17031.0
Florida,Miami-Dade,186809,3662,0,183147,2020-11-02 05:25:04,25.611236,-80.551706,12086.0
Texas,Harris,162306,2811,0,159495,2020-11-02 05:25:04,29.858649,-95.393395,48201.0
Arizona,Maricopa,159781,3606,0,156175,2020-11-02 05:25:04,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,41334,20634,57651,4665,10558,127,1157179,87432,27585,99576,...,992874,3082,66628,91589,1177,53075,10,2062,16415,8362
2020-10-31,41425,20875,57942,4756,10805,128,1166924,89813,27595,104925,...,1014793,3124,66932,92013,1180,53520,10,2063,16432,8367
2020-11-01,41501,21202,58272,4825,11035,128,1173533,92254,27601,109881,...,1038054,3149,67156,92325,1180,54060,10,2063,16480,8374


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,1533,502,1956,75,279,3,30792,1300,907,1082,...,46319,58,563,793,35,481,1,599,349,242
2020-10-31,1536,509,1964,75,284,3,31002,1341,907,1109,...,46645,58,566,798,35,483,1,599,349,243
2020-11-01,1536,518,1973,75,286,3,31140,1363,907,1130,...,46807,59,570,801,35,489,1,600,349,243


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,34258,11097,40014,3377,4107,116,961101,54316,25321,64987,...,2811,2532,63864,86335,1062,45896,8,1366,15600,7884
2020-10-31,34321,11189,40201,3475,4523,117,973939,55317,25336,67226,...,2839,2560,64235,86773,1062,46309,8,1366,15680,7894
2020-11-01,34326,11246,40395,3475,4920,117,985316,55827,25352,69336,...,2841,2601,64466,87156,1063,46773,8,1366,15733,7927


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2030,2048,2059,2074,2082,2103,2126,2141,2159,2173
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,474,490,496,506,508,516,523,525,530,534
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,2180,2262,2294,2313,2338,2371,2405,2437,2465,2743
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,837,850,858,861,869,877,885,899,916,941
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6517,6534,6546,6550,6571,6624,6649,6669,6702,6715


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-30,190496,15908,0,244045,110874,928028,104403,71207,24751,49,...,259488,924447,112932,2155,1362,179639,106573,23992,220091,13028
2020-10-31,192285,16355,0,245946,112190,932143,107327,71207,24951,49,...,260672,932261,114656,2179,1376,181190,107501,24460,225369,13298
2020-11-01,193985,16705,0,247473,113057,935878,109887,71207,25126,49,...,261426,936816,116510,2196,1376,182392,108315,24883,228862,13723


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-30        2141    6888    1056  866   2039     651   1020    4806   
2020-10-31        2159    6940    1060  873   2074     653   1022    4861   
2020-11-01        2173    6966    1061  878   2095     655   1024    4892   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-30         1381      769  ...     688    147      799      180   
2020-10-31         1389      783  ...     694    154      812      181   
2020-11-01         1392      787  ...     709    156      836      187   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-30            553   796   479          0      158    159  
2020-10-31            562   802   484          0      159    163  
2020-11-01            585   814   488          0      160    164  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-30,2932,81,0,5934,1900,17618,2278,4616,704,0,...,3341,18385,601,58,21,3643,2366,453,1972,87
2020-10-31,2967,82,0,5979,1925,17659,2285,4616,708,0,...,3353,18465,604,58,21,3654,2366,459,2031,87
2020-11-01,2973,83,0,5979,1958,17671,2288,4616,710,0,...,3353,18502,614,58,21,3655,2366,460,2047,87


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-30          31      71       9   15     25      17     41      63   
2020-10-31          31      71       9   15     25      17     41      65   
2020-11-01          31      71       9   15     25      17     41      65   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-30           47       15  ...       3      2        4        1   
2020-10-31           47       15  ...       3      2        4        1   
2020-11-01           47       15  ...       3      2        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-30              2     1     3         10        7      0  
2020-10-31              2     1     3         10        7      0  
2020-11-01              2     1     3         10        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,0.001599,0.015703,0.005564,0.021458,0.028143,0.024194,0.011697,0.028200,0.000218,0.059894,...,0.025213,0.012484,0.003555,0.003385,0.000000,0.009587,0.0,0.000000,0.005513,0.001557
2020-10-31,0.002202,0.011680,0.005048,0.019507,0.023395,0.007874,0.008421,0.027233,0.000363,0.053718,...,0.022076,0.013628,0.004563,0.004629,0.002549,0.008384,0.0,0.000485,0.001036,0.000598
2020-11-01,0.001835,0.015665,0.005695,0.014508,0.021286,0.000000,0.005664,0.027179,0.000217,0.047234,...,0.022922,0.008003,0.003347,0.003391,0.000000,0.010090,0.0,0.000000,0.002921,0.000837


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,0.000653,0.006012,0.003592,0.027397,0.014545,0.0,0.011497,0.022013,0.0,0.024621,...,0.005951,0.017544,0.003565,0.005070,0.0,0.016913,0.0,0.000000,0.002874,0.000000
2020-10-31,0.001957,0.013944,0.004090,0.000000,0.017921,0.0,0.006820,0.031538,0.0,0.024954,...,0.007038,0.000000,0.005329,0.006305,0.0,0.004158,0.0,0.000000,0.000000,0.004132
2020-11-01,0.000000,0.017682,0.004582,0.000000,0.007042,0.0,0.004451,0.016406,0.0,0.018936,...,0.003473,0.017241,0.007067,0.003759,0.0,0.012422,0.0,0.001669,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,0.000555,0.008177,0.009562,0.03589,0.099304,0.008696,0.015819,0.019885,0.003249,0.025582,...,0.004287,0.011182,0.004451,0.005099,0.000000,0.009857,0.0,0.0,0.000962,0.002543
2020-10-31,0.001839,0.008291,0.004673,0.02902,0.101290,0.008621,0.013358,0.018429,0.000592,0.034453,...,0.009961,0.011058,0.005809,0.005073,0.000000,0.008999,0.0,0.0,0.005128,0.001268
2020-11-01,0.000146,0.005094,0.004826,0.00000,0.087774,0.000000,0.011681,0.009220,0.000632,0.031387,...,0.000704,0.016016,0.003596,0.004414,0.000942,0.010020,0.0,0.0,0.003380,0.004180


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-30,0.007121,0.024868,NaN,0.006454,0.010591,0.004742,0.023649,0.010803,0.008064,0.0,...,0.010153,0.006233,0.020716,0.006539,0.006652,0.008171,0.009625,0.022415,0.023703,0.041657
2020-10-31,0.009391,0.028099,NaN,0.007790,0.011869,0.004434,0.028007,0.000000,0.008080,0.0,...,0.004563,0.008453,0.015266,0.011137,0.010279,0.008634,0.008708,0.019507,0.023981,0.020725
2020-11-01,0.008841,0.021400,NaN,0.006209,0.007728,0.004007,0.023852,0.000000,0.007014,0.0,...,0.002893,0.004886,0.016170,0.007802,0.000000,0.006634,0.007572,0.017294,0.015499,0.031960


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-30      0.007056  0.017730  0.000948  0.005807  0.014933  0.000000   
2020-10-31      0.008407  0.007549  0.003788  0.008083  0.017165  0.003072   
2020-11-01      0.006484  0.003746  0.000943  0.005727  0.010125  0.003063   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-30      0.000981  0.008604  0.000725  0.006545  ...  0.040847   
2020-10-31      0.001961  0.011444  0.005793  0.018205  ...  0.008721   
2020-11-01      0.001957  0.006377  0.002160  0.005109  ...  0.021614   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-30      0.035211  0.044444  0.028571   0.059387  0.021823  0.021322   
2020-10-31      0.047619  0.016270  0.005556   0.016275  0.007538  0.010438   
2020-11-01      0.012987  0.029557  0.033149   0.040925  0.014963  0.008264   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-30            NaN  0.032680  0.025806  
2020-10-31            NaN  0.006329  0.025157  
2020-11-01            NaN  0.006289  0.006135  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-30,0.006177,0.051948,NaN,0.002704,0.003168,0.003360,0.004409,0.001519,0.021771,NaN,...,0.023904,0.005964,0.005017,0.0,0.0,0.001925,0.002967,0.017978,0.012320,0.0
2020-10-31,0.011937,0.012346,NaN,0.007583,0.013158,0.002327,0.003073,0.000000,0.005682,NaN,...,0.003592,0.004351,0.004992,0.0,0.0,0.003019,0.000000,0.013245,0.029919,0.0
2020-11-01,0.002022,0.012195,NaN,0.000000,0.017143,0.000680,0.001313,0.000000,0.002825,NaN,...,0.000000,0.002004,0.016556,0.0,0.0,0.000274,0.000000,0.002179,0.007878,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-10-30         0.0  0.028986     0.0  0.0    0.0     0.0    0.0  0.016129   
2020-10-31         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.031746   
2020-11-01         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                     ... Wyoming                           \
Admin2         Chambers  Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-30          0.0  0.071429  ...     0.0    0.0      0.0      0.0   
2020-10-31          0.0  0.000000  ...     0.0    0.0      0.0      0.0   
2020-11-01          0.0  0.000000  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-30            0.0   0.0   0.0        0.0      0.0    NaN  
2020-10-31            0.0   0.0   0.0        0.0      0.0    NaN  
2020-11-01            0.0   0.0   0.0        0.0      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,0.002173,0.015141,0.005446,0.020282,0.024849,0.012253,0.011830,0.027413,0.000355,0.051586,...,0.025194,0.016025,0.003750,0.004020,0.001348,0.010115,4.081578e-41,0.000188,0.004056,0.001962
2020-10-31,0.002187,0.013411,0.005247,0.019895,0.024122,0.010064,0.010126,0.027323,0.000359,0.052652,...,0.023635,0.014826,0.004156,0.004325,0.001948,0.009250,2.040789e-41,0.000336,0.002546,0.001280
2020-11-01,0.002011,0.014538,0.005471,0.017201,0.022704,0.005032,0.007895,0.027251,0.000288,0.049943,...,0.023279,0.011414,0.003751,0.003858,0.000974,0.009670,1.020394e-41,0.000168,0.002733,0.001058


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,0.001663,0.008958,0.004030,0.018930,0.009842,1.254327e-60,0.011963,0.021255,0.000138,0.023766,...,0.006056,0.020796,0.004416,0.005354,1.379809e-19,0.015986,0.0,0.000013,0.001482,0.000741
2020-10-31,0.001810,0.011451,0.004060,0.009465,0.013881,6.271633e-61,0.009392,0.026397,0.000069,0.024360,...,0.006547,0.010398,0.004872,0.005829,6.899044e-20,0.010072,0.0,0.000007,0.000741,0.002437
2020-11-01,0.000905,0.014566,0.004321,0.004733,0.010462,3.135816e-61,0.006921,0.021401,0.000035,0.021648,...,0.005010,0.013820,0.005970,0.004794,3.449522e-20,0.011247,0.0,0.000838,0.000371,0.001218


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-30,0.000583,0.008712,0.005926,0.036568,0.057065,0.012640,0.015087,0.016343,0.001817,0.025143,...,0.007698,0.011838,0.004323,0.005468,0.000290,0.010186,2.391549e-43,0.000436,0.002235,0.002652
2020-10-31,0.001211,0.008501,0.005300,0.032794,0.079178,0.010630,0.014222,0.017386,0.001205,0.029798,...,0.008830,0.011448,0.005066,0.005270,0.000145,0.009592,1.195775e-43,0.000218,0.003682,0.001960
2020-11-01,0.000678,0.006798,0.005063,0.016397,0.083476,0.005315,0.012952,0.013303,0.000918,0.030592,...,0.004767,0.013732,0.004331,0.004842,0.000543,0.009806,5.978873e-44,0.000109,0.003531,0.003070


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-30,0.007180,0.024891,NaN,0.005695,0.009915,0.004736,0.020401,0.012597,0.007073,7.891928e-70,...,0.010074,0.006853,0.017972,0.007630,0.003715,0.007829,0.008339,0.019686,0.022681,0.035694
2020-10-31,0.008286,0.026495,NaN,0.006742,0.010892,0.004585,0.024204,0.006298,0.007577,3.945964e-70,...,0.007318,0.007653,0.016619,0.009384,0.006997,0.008231,0.008523,0.019596,0.023331,0.028209
2020-11-01,0.008563,0.023948,NaN,0.006475,0.009310,0.004296,0.024028,0.003149,0.007295,1.972982e-70,...,0.005105,0.006269,0.016394,0.008593,0.003499,0.007433,0.008048,0.018445,0.019415,0.030085


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-30      0.008171  0.010885  0.004021  0.006241  0.012441  0.000631   
2020-10-31      0.008289  0.009217  0.003905  0.007162  0.014803  0.001852   
2020-11-01      0.007387  0.006482  0.002424  0.006445  0.012464  0.002457   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-30      0.001959  0.010789  0.003260  0.006475  ...  0.034813   
2020-10-31      0.001960  0.011117  0.004526  0.012340  ...  0.021767   
2020-11-01      0.001958  0.008747  0.003343  0.008724  ...  0.021690   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-30      0.040180  0.040068  0.029139   0.045412  0.016001  0.019901   
2020-10-31      0.043899  0.028169  0.017347   0.030843  0.011769  0.015170   
2020-11-01      0.028443  0.028863  0.025248   0.035884  0.013366  0.011717   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-30      -0.999999  0.020486  0.038248  
2020-10-31      -0.999999  0.013407  0.031703  
2020-11-01      -0.999999  0.009848  0.018919  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-30,0.004798,0.050741,NaN,0.002403,0.006772,0.002979,0.005460,0.001450,0.011986,NaN,...,0.016505,0.005710,0.009658,2.285522e-28,7.629456e-07,0.003377,0.003772,0.015505,0.018680,0.036862
2020-10-31,0.008367,0.031544,NaN,0.004993,0.009965,0.002653,0.004266,0.000725,0.008834,NaN,...,0.010048,0.005031,0.007325,1.142761e-28,3.814728e-07,0.003198,0.001886,0.014375,0.024299,0.018431
2020-11-01,0.005195,0.021869,NaN,0.002497,0.013554,0.001666,0.002790,0.000362,0.005829,NaN,...,0.005024,0.003517,0.011941,5.713805e-29,1.907364e-07,0.001736,0.000943,0.008277,0.016089,0.009216


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-10-30      0.000312  0.014508  3.405980e-08  0.004503  0.000065   
2020-10-31      0.000156  0.007254  1.702990e-08  0.002251  0.000033   
2020-11-01      0.000078  0.003627  8.514949e-09  0.001126  0.000016   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-10-30      9.556401e-07  0.001563  0.010214  0.004443  0.035715  ...   
2020-10-31      4.778200e-07  0.000781  0.020980  0.002222  0.017857  ...   
2020-11-01      2.389100e-07  0.000391  0.010490  0.001111  0.008929  ...   

Province_State   Wyoming                                                    \
Admin2              Park    Platte      Sheridan Sublette Sweetwater Teton   
2020-10-30      0.000244  0.031250  8.526513e-14      0.0        0.0   0.0   
2020-10-31      0.000122  0.015625  4.263256e-14      0.0        0.0   0.0   
2020-11-01      0.000061  0.007812  2.131628e-14      0.0        0.0   0.0   

Province_State                                            
Admin2             Uinta Unassigned      Washakie Weston  
2020-10-30      0.001953  -0.007756  6.357829e-07    NaN  
2020-10-31      0.000977  -0.002599  3.178914e-07    NaN  
2020-11-01      0.000488  -0.001115  1.589457e-07    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201101,AK,16705,NaN,587852,NaN,totalTestsViral,604207,98.0,NaN,...,604557,1,0,0fe81cb4b3a17588cfde8bbd309c3f0f2e786fd2,0,0,0,0,0,NaN
1,20201101,AL,193985,28746.0,1187341,NaN,totalTestsViral,1352580,967.0,20450.0,...,1381326,6,0,0bd0f2572848cf6f46a651c076d32afceb8dc6a1,0,0,0,0,0,NaN
2,20201101,AR,112190,8708.0,1243490,NaN,totalTestsViral,1346972,639.0,7055.0,...,1355680,0,17,e0a88051a8b1e5499f93fcbf486088cd7219d109,0,0,0,0,0,NaN
3,20201101,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,...,1616,0,0,0c6d249303d6353496f3ea531770505166c65ec9,0,0,0,0,0,NaN
4,20201101,AZ,247473,5877.0,1544171,NaN,totalTestsPeopleViral,1785767,875.0,21558.0,...,1791644,2,66,041cb5bf71c1759f188bd5d7be53b9c298d703dc,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201101,AK,16705.0,NaN,587852.0,NaN,totalTestsViral,604207.0,98.0,NaN,...,604557,1,0,0fe81cb4b3a17588cfde8bbd309c3f0f2e786fd2,0,0,0,0,0,NaN
1,20201101,AL,193985.0,28746.0,1187341.0,NaN,totalTestsViral,1352580.0,967.0,20450.0,...,1381326,6,0,0bd0f2572848cf6f46a651c076d32afceb8dc6a1,0,0,0,0,0,NaN
2,20201101,AR,112190.0,8708.0,1243490.0,NaN,totalTestsViral,1346972.0,639.0,7055.0,...,1355680,0,17,e0a88051a8b1e5499f93fcbf486088cd7219d109,0,0,0,0,0,NaN
3,20201101,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,...,1616,0,0,0c6d249303d6353496f3ea531770505166c65ec9,0,0,0,0,0,NaN
4,20201101,AZ,247473.0,5877.0,1544171.0,NaN,totalTestsPeopleViral,1785767.0,875.0,21558.0,...,1791644,2,66,041cb5bf71c1759f188bd5d7be53b9c298d703dc,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-01,AK,16705,NaN,587852,NaN,totalTestsViral,604207,98.0,NaN,NaN,...,604557,1,0,0fe81cb4b3a17588cfde8bbd309c3f0f2e786fd2,0,0,0,0,0,NaN
2020-11-01,AL,193985,28746.0,1187341,NaN,totalTestsViral,1352580,967.0,20450.0,NaN,...,1381326,6,0,0bd0f2572848cf6f46a651c076d32afceb8dc6a1,0,0,0,0,0,NaN
2020-11-01,AR,112190,8708.0,1243490,NaN,totalTestsViral,1346972,639.0,7055.0,225.0,...,1355680,0,17,e0a88051a8b1e5499f93fcbf486088cd7219d109,0,0,0,0,0,NaN
2020-11-01,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,NaN,...,1616,0,0,0c6d249303d6353496f3ea531770505166c65ec9,0,0,0,0,0,NaN
2020-11-01,AZ,247473,5877.0,1544171,NaN,totalTestsPeopleViral,1785767,875.0,21558.0,222.0,...,1791644,2,66,041cb5bf71c1759f188bd5d7be53b9c298d703dc,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-01,AK,16705.0,NaN,587852.0,NaN,totalTestsViral,604207.0,98.0,NaN,NaN,...,604557,1,0,0fe81cb4b3a17588cfde8bbd309c3f0f2e786fd2,0,0,0,0,0,NaN
2020-11-01,AL,193985.0,28746.0,1187341.0,NaN,totalTestsViral,1352580.0,967.0,20450.0,NaN,...,1381326,6,0,0bd0f2572848cf6f46a651c076d32afceb8dc6a1,0,0,0,0,0,NaN
2020-11-01,AR,112190.0,8708.0,1243490.0,NaN,totalTestsViral,1346972.0,639.0,7055.0,225.0,...,1355680,0,17,e0a88051a8b1e5499f93fcbf486088cd7219d109,0,0,0,0,0,NaN
2020-11-01,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,NaN,...,1616,0,0,0c6d249303d6353496f3ea531770505166c65ec9,0,0,0,0,0,NaN
2020-11-01,AZ,247473.0,5877.0,1544171.0,NaN,totalTestsPeopleViral,1785767.0,875.0,21558.0,222.0,...,1791644,2,66,041cb5bf71c1759f188bd5d7be53b9c298d703dc,0,0,0,0,0,NaN
